Creating the Authors Template.


1. Create a new template called 'Authors' in your 'blog' app project and link it to a css file with the same name.
2. Use the auth.user model to show a list of all authors on this page!
    - from django.contrib.auth.models import User
3. Style the page as you desire.

In [ ]:
authors.html template:

In [ ]:
{% block content %}
<div class="container text-white-50 mt-5">
    <h1>Authors List</h1>
    <table class='table-bordered table-condensed'>
        <thead class='bg-dark'>
            <tr>
                <th>Username</th>
                <th>Email</th>
                <th>Date of Creation</th>
            </tr>
        </thead>
        <tbody>
            {% for author in authors %}
                <tr>
                    <td>{{ author.username }}</td>
                    <td>{{ author.email }}</td>
                    <td>{{ author.date_joined|date:"F j, Y, g:i a" }}</td>
                </tr>
            {% endfor %}
        </tbody>
    </table>
</div>
{% endblock content %}

authors.css file:

In [ ]:
table {
    width: 100%;
    border-collapse: collapse;
}
th, td {
    padding: 10px;
    text-align: left;
}
tr{
    cursor: pointer;
}
tr:hover{
    background-color:rgba(255,255,255,0.6);
    color:black;
    border:1px solid black;
}

AuthorsView class:

from django.contrib.auth.models import User
class AuthorsView(ListView):
    template_name = 'blog/authors.html'
    model = User
    context_object_name = 'authors'

We will come back here to later to modify a few things.<br>

Lets move on and create a new model called tags:<br>
The model has to have the following attributes:<br>
- caption = models.CharField(max_length=20)

In [ ]:
class Tag(models.Model):
    caption = models.CharField(max_length=20)

    def __str__(self):
        return self.caption

    class Meta:
        verbose_name_plural = 'All Tags'

Now create a new model called posts with the same attributes as our 'website' project<br>

In [ ]:
class Post(models.Model):
    title = models.CharField(max_length=200, null=False)
    slug = models.SlugField(unique=True, db_index=True)
    date = models.DateField(auto_now=True)
    image = models.ImageField(upload_to='images', null=True)
    excerpt = models.CharField(max_length=300)
    content = models.TextField(validators=[MinLengthValidator(10)])
    author = models.ForeignKey(User, null=True, on_delete=models.SET_NULL, related_name='posts')
    tags = models.ManyToManyField('Tag')

    def __str__(self):
        return f"{self.title}"

    def save(self, *args, **kwargs):
        self.slug = slugify(self.title)
        super().save(*args, **kwargs)

    class Meta:
        verbose_name_plural = 'All Posts'

Make the needed changes to the admin.py file so you can access both Post and Tag models.

In [ ]:
from django.contrib import admin
from .models import Post, Tag


class TagAdmin(admin.ModelAdmin):
    list_display = ['caption']

class PostAdmin(admin.ModelAdmin):
    prepopulated_fields = {'slug':('title',)}
    list_display = ['title', 'date', 'author']
    list_filter = ('author', 'date', 'tags')

admin.site.register(Tag, TagAdmin)
admin.site.register(Post, PostAdmin)

You need to install pillow package for handleing images.

In [ ]:
python -m pip install --upgrade pillow

Makemigrations and migrate

In [ ]:
python manage.py makemigrations
python manage.py migrate

Now, we need fix he archives template, write the needed code to present a list of all posts in that tempalte.its very simple! you need to create a view for it too.<br>

ArchiveView class:

In [ ]:
class ArchiveView(ListView):
    template_name = 'blog/archive.html'
    model = Post
    context_object_name = 'allPosts'
    def get_queryset(self):
        return Post.objects.all().order_by('-date')

archive.html template:

{% block content %}
    <div class="container text-white-50 mt-5">
        {% if allPosts %}
        <h1>Posts Archive</h1>
        <table class='table-bordered table-condensed'>
            <thead class='bg-dark'>
                <tr>
                    <th>Title</th>
                    <th>Tags</th>
                    <th>Date</th>
                    <th>Author</th>
                </tr>
            </thead>
            {% for post in allPosts %}
                <tr>
                    <td>{{post.title}}</td>
                    <td>{{post.tags}}</td>
                    <td>{{post.date}}</td>
                    <td>{{post.author}}</td>
                </tr>
            {% endfor %}
        </table>
        {% else %}
            <h1>No posts yet.</h1>
        {% endif %} 
    </div>
    
{% endblock content %}

archive.css file:

In [ ]:
table {
    width: 100%;
    border-collapse: collapse;
}
th, td {
    padding: 10px;
    text-align: left;
}
tr{
    cursor: pointer;
}
tr:hover{
    background-color:rgba(255,255,255,0.6);
    color:black;
    border:1px solid black;
}

Now we need to add a new link to the navbar (only visible for authenticated users) that takes the users to a template where they acan create a enw post.<br>

In [ ]:
{% if user.is_authenticated %}
    <li class="nav-item">
        <a class="btn btn-primary text-white mr-2" href="{% url "compose-new-post-url" %}"><i class="bi bi-pencil-square"></i></a>
    </li>
{% endif %}

Create a file called forms.py:

In [ ]:
from django import forms
from.models import Post


class ComposeForm(forms.ModelForm):
    class Meta:
        model = Post
        fields = ['title', 'image', 'excerpt', 'content', 'tags']
        labels = {
            "title":"Title",
            "excerpt":"Short overview",
            "content":"Text",
            'tags':'Tags'
        }

urls.py file:

In [ ]:
urlpatterns = [
    path('', views.IndexView.as_view(), name='blog-root-url'),
    path('archive', views.ArchiveView.as_view(), name='blog-archive-url'),
    path('authors', views.AuthorsView.as_view(), name='blog-authors-url'),
    path('about', views.AboutView.as_view(), name='blog-about-url'),
    path('compose', views.ComposeView.as_view(), name='compose-new-post-url')
]

ComposeView class:

In [ ]:
class ComposeView(CreateView):
    template_name = 'blog/compose.html'
    model = Post
    form_class = ComposeForm
    success_url = reverse_lazy('blog-root-url')

    def form_valid(self, form):
        form.instance.author = self.request.user
        print("validated")
        return super().form_valid(form)

compose.html template:

<div class="d-flex justify-content-center text-white-50 mt-5">
    <div class="col-md-3 bg-dark p-4 rounded-lg">
        <h1 class='text-center text-white'>Compose New Post</h1>
        <form action="" method="POST" enctype='multipart/form-data'>
            {% csrf_token %}
            {{ form }}
            <button class='btn btn-primary mt-4 btn-block' type="submit">Publish</button>
        </form>
    </div>
</div>

The other changes are need to be done by the students!<br>

End of Chapter 12